<a href="https://colab.research.google.com/github/fatsmcgee/AssortedDataScience/blob/master/Cuda_Numba_Matmul_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Matmult implementations following: https://siboehm.com/articles/22/CUDA-MMM

In [1]:
import numpy as np
from numba import cuda, float32
import math


In [2]:
# Helper function to calculate grid dimensions
def ceil_div(a, b):
    return math.ceil(a / b)

In [3]:
@cuda.jit
def sgemm_naive(M, N, K, alpha, A, B, beta, C):
    # Get thread position
    x, y = cuda.grid(2)  # Shortcut for 2D grid position

    # Check if thread is within matrix bounds
    if x < M and y < N:
        tmp = 0.0
        for i in range(K):
            tmp += A[x, i] * B[i, y]

        # C = α*(A@B) + β*C
        C[x, y] = alpha * tmp + beta * C[x, y]

In [9]:
# A is MxK, B is KxN, C is MxN
M,N,K = (1024, 1024, 1024)

# Create random input matrices (np.random.rand accepts dtype parameter)
A = np.random.rand(M, K).astype(np.float32)
B = np.random.rand(K, N).astype(np.float32)
C = np.random.rand(M, N).astype(np.float32)

# Transfer to device and launch kernel in one go using auto-managed memory
threadsperblock = (32, 32)
blockspergrid = (ceil_div(M, threadsperblock[0]), ceil_div(N, threadsperblock[1]))

sgemm_naive[blockspergrid, threadsperblock](
    M, N, K,
    float32(1.0),  # alpha
    A, B,
    float32(0.0),  # beta
    C
)

# Verify (C is automatically updated due to managed memory)
#np.testing.assert_allclose(C, A @ B, rtol=1e-5)

/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:888: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
